In [2]:
import pandas as pd
import numpy as np
import os 
from itertools import product
from tqdm.notebook import tqdm
import statsmodels.api as sm
from multiprocessing import Pool
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings("ignore")
# from evaluate_func import *

In [3]:
market_data_train = pd.read_pickle('/code/20211027_alldays_market_data_cleanadd/market_data_train.pkl')


# 输入数据标准化

In [10]:
# def normalize_time(x):

def second_to_time(second,add = True):       # 跳过11.30-13.00，分时，原始数据的时间是1e7的形式
    if add==False:
        if second>41400:
            second += 3600*1.5
    m, s = divmod(second, 60)
    h, m = divmod(m, 60)
    return h*1e7 + m*1e5 + s*1e3

def time_to_second(time):
    h = time // 1e7
    min = time%1e7//1e5
    s = time % 1e5 /1e3
    result = 3600*h + 60*min+s
    if result>=46800:      # 13点以后的时间，转成11.30之后的时间，让上下午的时间连起来
        result -= 1.5*3600
    return result
    

In [16]:
def time_to_second(time):
    h = time // 1e7
    min = time%1e7//1e5
    s = time % 1e5 /1e3
    result = 3600*h + 60*min+s
    if result>41400:      # 13点以后的时间，转成11.30之后的时间，让上下午的时间连起来
        result -= 1.5*3600
    return result - 34200
# time_to_second(11.29e7)

7140.0

In [249]:
# market_data_train[(20191204, 1)]
data[np.isnan(data[:,24])]
np.argwhere(np.isnan(data[:,23]))   # 每天的最后几个数据有缺失

array([[4740],
       [4741],
       [4742],
       [4743],
       [4744],
       [4745],
       [4746],
       [4747],
       [4748],
       [4749],
       [4750],
       [4751],
       [4752],
       [4753],
       [4754],
       [4755],
       [4756],
       [4757],
       [4758],
       [4759]])

In [279]:
date = 20191204 # 日期参数
code = 1        # 股票
time_window_ofi = 10    # ofi的时间窗口长度,初始10s

def ofi_calc(data):
    OFI =[]                      # 返回一个长度为5的各个档位OFI
    for m in range(1,6):        # 第m档数据
        
        e = []

        for tal in range(1, data.shape[0]):    
            # W
            if data[tal,4+m] > data[tal-1,4+m]:   # if b_m(tal_n) > b_m(tal_n-1) 
                w = data[tal, 14+m]
            elif data[tal,4+m] == data[tal-1,4+m]:     # if b_m(tal_n) == b_m(tal_n-1) 
                w = data[tal, 14+m] - data[tal-1, 14+m]
            else:                                            # # if b_m(tal_n) < b_m(tal_n-1)                        
                w = data[tal-1, 14+m]

            # V
            if data[tal, 9+m] > data[tal-1, 9+m]:   # if a_m(tal_n) > a_m(tal_n-1) 
                v = -data[tal-1, 19+m]
            elif data[tal,4+m] == data[tal-1,4+m]:     # if a_m(tal_n) == a_m(tal_n-1) 
                v = data[tal, 19+m] - data[tal-1, 19+m]
            else:                                            # # if a_m(tal_n) < a_m(tal_n-1)                        
                v = data[tal, 19+m]

           
            e.append(w-v)
        
        OFI.append(np.sum(e))
    return OFI

class input_data:
    def __init__(self, date, code, k=60, d=40, n=1, time_window_ofi=10):
        self.date = date
        self.code = code

        self.time_window_ofi = time_window_ofi     # 计算ofi的时间窗口长度,默认是十秒
        self.k = k                                 # 历史k时间段计算价格波动作为上下界阈值
        self.d = d                                 # 持仓时间阈值                                     
        self.n = n                                 # n倍价格波动作为阈值

        self.data = market_data_train[(date, code)]

    def normalized_input_data(self):
        time_window_ofi = self.time_window_ofi
        data = self.data

        diff_data = np.diff(data,axis=0)[:,[3,4,5,10,15,20]]     # 差分数据：保留成交量，成交金额，买一价+量，卖一价+量六个差分数据
        ask1_bid1_delta = (data[1:,10] - data[1:,5]).reshape(-1,1)                 # 卖一价减去买一价
        input_8 = np.hstack((data[1:,3].reshape(-1,1), diff_data,ask1_bid1_delta))  # 除了ofi以外的其他数据 

        second_time = pd.Series(data[:,1]).apply(time_to_second)    # 把时间转化成秒，且上下午的时间是连起来的

        ofi_list = []
        for t in second_time:
            ofi = ofi_calc(data[(second_time>t-time_window_ofi) & (second_time<=t)])      # 这个时间段的5档ofi
            ofi_list.append(ofi)       # 时间序列上的五档ofi结果
        ofi_list = np.array(ofi_list)[1:]

        input_result_array = np.hstack((input_8, ofi_list))

        return input_result_array[1:]     # 输出1：长度是为了和trple barrier的长度一致

    def triple_barrier_labeling(self):
        k =self.k
        d =self.d
        n =self.n
        data = self.data

        # mid_price = pd.Series(data[:,26])     # 将中间价格转化成series，方便后续的滚动窗口操作,固定slcie数量，即窗口长度，**todo**
        
        # 计算每个时间点前k个时间点之内的价格波动率
        return_data = np.diff(data[:,26]) / data[1:,26]    # 收益率array
        second_time = pd.Series(data[:,1]).apply(time_to_second)    # 把时间转化成秒，且上下午的时间是连起来的
        
        volitility_list = []                   # 窗口为K的历史收益波动率序列第一个值肯定是0,其长度为data长度-1
        for t in second_time[1:]:
            volitility = return_data[(second_time[1:]>t-k) & (second_time[1:]<=t)].std()
            volitility_list.append(volitility)
        
        volitility_array = np.array(volitility_list)
        
        # 从第二个return_data开始计算是否触碰边界,即第三个midprice时间开始计算label
        up_list = list((1 + volitility_array[1:]) * data[2:,26])          # 价格上界
        down_list = list((1 - volitility_array[1:]) * data[2:,26])        # 价格下界

        label_list = []
        for n, t in enumerate(second_time[2:]):
            price_tmp = data[2:,26][(second_time[2:]>t) & (second_time[2:]<=t + d)]   # 对每个时间点，price_tmp为其往后d时间内的价格
            label_array = np.where( price_tmp> up_list[n], 1, np.where(price_tmp < down_list[n], -1, 0))
            label = label_array[label_array!=0][0] if len(label_array[label_array!=0]) > 0 else 0
            label_list.append(label)

        return np.array(label_list).reshape(-1,1)
    def input_data_n_label(self):
        result = np.hstack((self.normalized_input_data(), self.triple_barrier_labeling()))
        result = result[~np.isnan(result).any(axis=1)]     # 去掉inputdata缺失的数据
        return result

In [277]:
# check one day one stock

aa = input_data(20191204,1)
# aa.normalized_input_date()
# return_data, second_time = aa.triple_barrier_labeling()
# # return_data[]
# k=60
# for t in second_time[1:]:
#     return_data[(second_time[1:]>t-k) & (second_time[1:]<=t)]
# volitility_array = np.array(aa.triple_barrier_labeling())
# volitility_array + 1

# label = aa.triple_barrier_labeling()
# input_data_code1 = aa.normalized_input_data()

train = aa.input_data_n_label()
# 一个股票一天生成输入变量和label需要10s左右

In [275]:
np.hstack((input_data_code1, np.array(label).reshape(-1,1))).shape

(4759, 14)

In [264]:
np.argwhere(np.isnan(input_data_code1).any(axis=1))
# input_data_code1[~np.isnan(input_data_code1).any(axis=1),:].shape
# input_data_code1.shape

array([[4738],
       [4739],
       [4740],
       [4741],
       [4742],
       [4743],
       [4744],
       [4745],
       [4746],
       [4747],
       [4748],
       [4749],
       [4750],
       [4751],
       [4752],
       [4753],
       [4754],
       [4755],
       [4756],
       [4757]])

In [84]:
# check tmp
date = 20191204 # 日期参数
code = 1        # 股票
# time_to_second(market_data_train[(date, code)][:,1])
data = market_data_train[(date, code)]
second_time = pd.Series(data[:,1]).apply(time_to_second)    # 吧时间转化成秒，且上下午的时间是连起来的
time_window_ofi = 10

diff_data = np.diff(data,axis=0)[:,[3,4,5,10,15,20]]     # 差分数据：保留成交量，成交金额，买一价+量，卖一价+量六个差分数据
ask1_bid1_delta = (data[1:,10] - data[1:,5]).reshape(-1,1)                 # 卖一价减去买一价
input_8 = np.hstack((data[1:,3].reshape(-1,1), diff_data,ask1_bid1_delta))  # 除了ofi意外的其他数据

# np.diff(data[:10],axis=0).shape

# data = market_data_train[(date, code)]
# diff_data = np.diff(data,axis=0)[:,[3,4,5,10,15,20]]     # 差分数据：保留成交量，成交金额，买一价+量，卖一价+量六个差分数据

ofi_list = []
for t in second_time:
    ofi = ofi_calc(data[(second_time>t-time_window_ofi) & (second_time<=t)])      # 这个时间段的5档ofi
    ofi_list.append(ofi)       # 时间序列上的五档ofi结果

ofi_list = np.array(ofi_list)[1:]
        # return ofi_list

input_result_array = np.hstack((input_8, ofi_list))

In [207]:
len(list(market_data_train.keys()))             # 所有数据共42353个

42353

In [204]:
input_and_label = {}
for x in tqdm(list(market_data_train.keys())):
    date = x[0]
    code = x[1]
    input_and_label[x] = input_data(date, code)#[input_data(date, code).normalized_input_data(), input_data(date, code).triple_barrier_labeling()]
    

In [233]:
date_list = np.unique([x[0] for x in list(market_data_train.keys())])   # 20191204 - 2020_01_31之前的数据
train_date = date_list[:int(len(date_list)/2)]     # 训练集日期
test_date = date_list[int(len(date_list)/2):]     # 测试集日期

In [224]:
x_before20201231 = [x for x in list(market_data_train.keys()) if x[0] <= 20200131]   # 20191204 - 2020_01_31之前的数据
np.unique([y[0] for y in x_before20201231])      # 24天--训练集


array([20191204, 20191205, 20191206, 20191209, 20191210, 20191211,
       20191213, 20191216, 20191219, 20191225, 20191227, 20191231,
       20200102, 20200103, 20200106, 20200108, 20200109, 20200113,
       20200114, 20200115, 20200116, 20200120, 20200121, 20200123])

In [217]:
len(np.unique([y[1] for y in x_before20201231]))      # 400只股票


400

In [223]:
x_between_20201231_20200301 = [x for x in list(market_data_train.keys()) if x[0] >= 20200131 and x[0] < 20200320]   # 20200131 - 20200320
np.unique([y[0] for y in x_between_20201231_20200301])      # 24天--测试集


array([20200203, 20200206, 20200207, 20200210, 20200211, 20200213,
       20200214, 20200217, 20200218, 20200219, 20200220, 20200221,
       20200225, 20200227, 20200228, 20200305, 20200306, 20200309,
       20200310, 20200311, 20200313, 20200316, 20200318, 20200319])

In [236]:
# 生成code1的训练集和测试集
train_data_code1_index = [x for x in list(market_data_train.keys()) if x[0] in train_date and x[1] ==1]
test_data_code1_index = [x for x in list(market_data_train.keys()) if x[0] in test_date and x[1] ==1]

In [280]:
# input_and_label = {}
def map_net_prepare(x):
# for x in tqdm(list(market_data_train.keys())):
    date = x[0]
    code = x[1]
    result = input_data(date, code).input_data_n_label()#[input_data(date, code).normalized_input_data(), input_data(date, code).triple_barrier_labeling()]
    return result

with Pool(32) as pool:  
    # train_data = pool.map(map_net_prepare, x_before20201231)
    train_data_code1 = pool.map(map_net_prepare, train_data_code1_index)    # code==1的训练集数据
    test_data_code1 = pool.map(map_net_prepare, test_data_code1_index)      # code==1的测试集数据

In [230]:
pd.to_pickle(train_data, 'train_input_n_label.pkl')

In [282]:
pd.to_pickle(train_data_code1, 'train_data_code1.pkl')
pd.to_pickle(test_data_code1, 'test_data_code1.pkl')
